In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
import seaborn as sns
sns.set()

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# You can configure the format of the images: ‘png’, ‘retina’, ‘jpeg’, ‘svg’, ‘pdf’.
%config InlineBackend.figure_format = 'svg'
# this statement allows the visuals to render within your Jupyter Notebook
%matplotlib inline 

# Data Cleaning & Feature Engineering

In [3]:
df = pd.read_csv('../data/KaggleV2-May-2016.csv')

In [4]:
df

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110522,2.572134e+12,5651768,F,2016-05-03T09:15:35Z,2016-06-07T00:00:00Z,56,MARIA ORTIZ,0,0,0,0,0,1,No
110523,3.596266e+12,5650093,F,2016-05-03T07:27:33Z,2016-06-07T00:00:00Z,51,MARIA ORTIZ,0,0,0,0,0,1,No
110524,1.557663e+13,5630692,F,2016-04-27T16:03:52Z,2016-06-07T00:00:00Z,21,MARIA ORTIZ,0,0,0,0,0,1,No
110525,9.213493e+13,5630323,F,2016-04-27T15:09:23Z,2016-06-07T00:00:00Z,38,MARIA ORTIZ,0,0,0,0,0,1,No


In [5]:
df.describe()

,PatientId,AppointmentID,Age,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received
count,1.105270e+05,1.105270e+05,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000,110527.000000
mean,1.474963e+14,5.675305e+06,37.088874,0.098266,0.197246,0.071865,0.030400,0.022248,0.321026
std,2.560949e+14,7.129575e+04,23.110205,0.297675,0.397921,0.258265,0.171686,0.161543,0.466873
min,3.921784e+04,5.030230e+06,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,4.172614e+12,5.640286e+06,18.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,3.173184e+13,5.680573e+06,37.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,9.439172e+13,5.725524e+06,55.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,9.999816e+14,5.790484e+06,115.000000,1.000000,1.000000,1.000000,1.000000,4.000000,1.000000


In [6]:
df.info() ### check dtypes and for missing values 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 110527 entries, 0 to 110526
Data columns (total 14 columns):
PatientId         110527 non-null float64
AppointmentID     110527 non-null int64
Gender            110527 non-null object
ScheduledDay      110527 non-null object
AppointmentDay    110527 non-null object
Age               110527 non-null int64
Neighbourhood     110527 non-null object
Scholarship       110527 non-null int64
Hipertension      110527 non-null int64
Diabetes          110527 non-null int64
Alcoholism        110527 non-null int64
Handcap           110527 non-null int64
SMS_received      110527 non-null int64
No-show           110527 non-null object
dtypes: float64(1), int64(8), object(5)
memory usage: 11.8+ MB


In [7]:
df.AppointmentID.value_counts() ### all appointment ids are unique! 

5769215    1
5731652    1
5707080    1
5702986    1
5715276    1
          ..
5586290    1
5584243    1
5598584    1
5602682    1
5771266    1
Name: AppointmentID, Length: 110527, dtype: int64

In [8]:
###remove negative value for age 
df['Age'].replace(-1,np.NaN, inplace=True)

In [9]:
### Cast target and gender feature as binaries
### Yes = 1
### F = 1
df['No-show'] = pd.get_dummies(df['No-show'])
df['No-show'] = pd.get_dummies(df['No-show']) ### repeating because I want Yes to be '1'
df['Gender'] = pd.get_dummies(df['Gender'])

In [10]:
### Split time stamp from date for date time features 

df['ScheduledDay_Day'] = df['ScheduledDay'].str.split(pat='T', expand = True)[0]
df['AppointmentDay_Day'] = df['AppointmentDay'].str.split(pat='T', expand = True)[0]

df['AppointmentDay_Time'] = df['AppointmentDay'].str.split(pat='T', expand = True)[1]
df['ScheduledDay_Time'] = df['ScheduledDay'].str.split(pat='T', expand = True)[1]

In [11]:
df['AppointmentDay_Time'].value_counts() ### thre are no time stamps for appointment day 

00:00:00Z    110527
Name: AppointmentDay_Time, dtype: int64

In [12]:
### remove the trailing Z
df['ScheduledDay_Time'] = df['ScheduledDay_Time'].str.translate({ord('Z'): None})

In [13]:
### Create feature for hour of the day for scheduled 
df['ScheduledDay_Hours'] = df['ScheduledDay_Time'].apply(lambda x:x[0:2])

In [14]:
### Cast datetime features  
df['ScheduledDay_Day'] = pd.to_datetime(df['ScheduledDay_Day'], infer_datetime_format=True)
df['AppointmentDay_Day'] = pd.to_datetime(df['AppointmentDay_Day'], infer_datetime_format=True)

In [15]:
df['ScheduledDay_Date'] = df['ScheduledDay_Day'].astype(str).apply(lambda x:x[-2:])
df['AppointmentDay_Date'] = df['AppointmentDay_Day'].astype(str).apply(lambda x:x[-2:])

In [16]:
### Calculate time lag between ScheduleDay and AppointmentDay
df['day_difference'] = (df['AppointmentDay_Day'] - df['ScheduledDay_Day'])

In [17]:
df['day_difference'].describe() ### there seems to be negative date differences...

count                     110527
mean     10 days 04:24:31.828602
std      15 days 06:07:11.673762
min            -6 days +00:00:00
25%              0 days 00:00:00
50%              4 days 00:00:00
75%             15 days 00:00:00
max            179 days 00:00:00
Name: day_difference, dtype: object

In [18]:
### -1 means less than a day, but when I checked the entries for time differences
### of less than -1, the scheduled date is clearly after appointment date 
### I suspect that the negative days are arising from data entry errors, 
### where the scheduled and appointment days were mixed up

In [19]:
df['day_difference'] = df['day_difference'].apply(lambda x: x.days) ### cast day lag into absolute values
df['day_difference']

0          0
1          0
2          0
3          0
4          0
          ..
110522    35
110523    35
110524    41
110525    41
110526    41
Name: day_difference, Length: 110527, dtype: int64

In [20]:
df.Neighbourhood.value_counts()

### there are way too many variables to create dummies.
### can't really find additional info about the neigbourhoods to append 

JARDIM CAMBURI                 7717
MARIA ORTIZ                    5805
RESISTÊNCIA                    4431
JARDIM DA PENHA                3877
ITARARÉ                        3514
                               ... 
ILHA DO BOI                      35
ILHA DO FRADE                    10
AEROPORTO                         8
ILHAS OCEÂNICAS DE TRINDADE       2
PARQUE INDUSTRIAL                 1
Name: Neighbourhood, Length: 81, dtype: int64

In [21]:
### This feature is suppose to be true/false 
df['Handcap'].replace([2,3,4],1, inplace = True)

In [22]:
### create a feature that tells us if its a repeat patient 

patient_visits = df['PatientId'].value_counts()

In [23]:
### let's create a new feature to tell us how many prior appointments 
total_appointments = df.groupby('PatientId').AppointmentID.count().reset_index()
total_appointments.columns = ['PatientId','total_appointments']

In [24]:
df = df.merge(total_appointments, how = 'left', on = 'PatientId')

In [25]:
### we can also create a flag for regular patients 
### we can change the treshold later 
df['regular_patient'] = np.where(df['total_appointments']>1,1,0)

In [26]:
### yay we can also check the day of the week for appointments and schedule 

df['appointment_weekday'] = df['AppointmentDay_Day'].apply(lambda x:x.isoweekday())
df['schedule_weekday'] = df['ScheduledDay_Day'].apply(lambda x:x.isoweekday())

In [27]:
df['appointment_weekday'].value_counts() ### some sunday appointments... are these special cases?

3    25867
2    25640
1    22715
5    19019
4    17247
6       39
Name: appointment_weekday, dtype: int64

In [28]:
df['schedule_weekday'].value_counts() ### hmmm, looks like need to make appointment at the clinic 

2    26168
3    24262
1    23085
5    18915
4    18073
6       24
Name: schedule_weekday, dtype: int64

In [29]:
### let's more sunday entries, they seem to be special cases 
df['appointment_weekday'].replace(6,np.NaN, inplace=True)
df['schedule_weekday'].replace(6,np.NaN, inplace=True)

In [30]:
### create feature for no. of afflictions 

df['total_conditions'] = df['Hipertension']+df['Diabetes']+df['Handcap']+df['Alcoholism']

In [31]:
df['total_conditions'].value_counts()

0    84115
1    18123
2     7658
3      618
4       13
Name: total_conditions, dtype: int64

In [32]:
df = df.drop(['PatientId','ScheduledDay','AppointmentDay','ScheduledDay_Time','AppointmentID','AppointmentDay_Time'], axis =1)

In [42]:
def plot_features(df, sample_size=500):
    
    sample = df.sample(1000, random_state=44)
    sns.pairplot(sample,hue='No-show', plot_kws=dict(alpha=.3, edgecolor='none'))

plot_features(df)

Error in callback <function flush_figures at 0x11c33e0e0> (for post_execute):


KeyboardInterrupt: 

In [ ]:
def get_user_split_data(df, test_size=.2, seed=42):

    rs = np.random.RandomState(seed)
    
    total_users = df['user_id'].unique() 
    test_users = rs.choice(total_users, 
                           size=int(total_users.shape[0] * test_size), 
                           replace=False)

    df_tr = df[~df['user_id'].isin(test_users)]
    df_te = df[df['user_id'].isin(test_users)] 

    y_tr, y_te = df_tr['in_cart'], df_te['in_cart']
    X_tr = df_tr.drop(['product_id','user_id','latest_cart','in_cart'],axis=1) 
    X_te = df_te.drop(['product_id','user_id','latest_cart','in_cart'],axis=1)

    return X_tr, X_te, y_tr, y_te

In [ ]:
def plot_features(df, sample_size=500):
    
    sample = (df.drop(['product_id','user_id','latest_cart'],axis=1)
                .sample(1000, random_state=44)) 
    sns.pairplot(sample,hue='in_cart', plot_kws=dict(alpha=.3, edgecolor='none'))

plot_features(df_X)